In [ ]:
!pip install tensorflow==2.4.0
!pip install keras==2.4.0
!pip install kymatio
!pip install tqdm
!pip install iterative-stratification
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.7 MB 16 kB/s 
     |████████████████████████████████| 14.8 MB 38.8 MB/s 
     |████████████████████████████████| 132 kB 67.5 MB/s 
     |████████████████████████████████| 3.8 MB 44.6 MB/s 
     |████████████████████████████████| 462 kB 43.2 MB/s 
     |████████████████████████████████| 2.9 MB 25.2 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68725 sha256=edee3453e16384160e52ea4472db16a6129317a4ad011f5f89cb7348af432c10
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: numpy
    Found existing installation: n

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 170 kB 5.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.9 MB/s 


In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
from absl import app, flags
from easydict import EasyDict
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle
import sys
sys.path.append("drive/MyDrive/Scattering_Novo/src")
from DataHandler import DataHandler
from ModelHandler import ModelHandler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

 
configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": True,
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "drive/MyDrive/YOLO_NILM/Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "drive/MyDrive/Scattering_Novo/DeepDFML/Final/", 
    "FOLDER_DATA_PATH": "drive/MyDrive/Scattering_Novo/DeepDFML/Final/", 
    "N_EPOCHS_TRAINING": 5000,
    "PERCENTUAL": [1],
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None # Nível de ruído em db
}

def freeze(model, task_name='classification'):
    for layer in model.layers:
        if task_name in layer.name:
            layer.trainable = True
        else:
            layer.trainable = False

    for layer in model.layers:
        print(layer.name, layer.trainable)

    return model

def calculating_class_weights(y_true):
    '''
        Source: https://stackoverflow.com/questions/48485870/multi-label-classification-with-class-weights-in-keras
    '''
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced', classes=[0.,1.], y=y_true[:, i])
    return weights


def reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,percentual):
    import numpy as np
    max_index = int(percentual*X_all.shape[0])
    np.random.seed(100)
    index = np.random.randint(max_index,size=(max_index-1))
    X_all = X_all[index]
    ydet_all = ydet_all[index]
    ytype_all = ytype_all[index]
    yclass_all = yclass_all[index]

    return X_all,ydet_all,ytype_all,yclass_all

ngrids = configs["N_GRIDS"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
trainSize = configs["TRAIN_SIZE"]
folderDataPath = configs["FOLDER_DATA_PATH"]
 
dataHandler = DataHandler(configs)

if not os.path.isfile(folderDataPath + "sorted_aug_data_5_12800.p"):
    print("Sorted data not found, creating new file...")
    x, ydet, yclass, ytype, ygroup = dataHandler.loadData(hand_augmentation=configs["USE_HAND_AUGMENTATION"], SNR=configs["SNRdb"])
    print("Data loaded")

    data_mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    strat_classes = np.max(yclass, axis=1)
    train_index, test_index = next(data_mskf.split(x, strat_classes))

    y_train = {
        "detection": ydet[train_index], 
        "type": ytype[train_index], 
        "classification": yclass[train_index], 
        "group": ygroup[train_index]
    }
    
    y_test = {
        "detection": ydet[test_index], 
        "type": ytype[test_index], 
        "classification": yclass[test_index], 
        "group": ygroup[test_index]
    }
    
    dict_data = {
        "x_train": x[train_index], 
        "x_test": x[test_index], 
        "y_train": y_train, 
        "y_test": y_test
    }

    print("Data sorted")

    try:
        os.mkdir(folderDataPath)
    except:
        pass

    pickle.dump(dict_data, open(folderDataPath + "sorted_aug_data_5_12800.p", "wb"))
    print("Data stored")
else:
    dict_data = pickle.load(open(folderDataPath + "sorted_aug_data_5_12800.p", "rb"))



modelHandler = ModelHandler(configs)
 
X_all = dict_data["x_train"]
ydet_all = dict_data["y_train"]["detection"]
ytype_all = dict_data["y_train"]["type"]
yclass_all = dict_data["y_train"]["classification"]

if configs["PERCENTUAL"][0]!=1:
        X_all,ydet_all,ytype_all,yclass_all = reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,configs["PERCENTUAL"][0])

print(X_all.shape)
print(dict_data["x_test"].shape)


Using TensorFlow backend.


(16649, 16640, 1)
(4163, 16640, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, GlobalAveragePooling1D, Flatten, MaxPool1D, GlobalMaxPooling1D, BatchNormalization
from tensorflow.keras.models import Model
from kymatio.keras import Scattering1D



In [ ]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph


def get_flops(model, batch_size=None):
    if batch_size is None:
        batch_size = 1

    real_model = tf.function(model).get_concrete_function(tf.TensorSpec([batch_size] + model.inputs[0].shape[1:], model.inputs[0].dtype))
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(real_model)

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                            run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops

def get_flops_output(model, batch_size=None):
    if batch_size is None:
        batch_size = 1
    argument = (tf.TensorSpec([batch_size] + model.inputs[0].shape[1:], model.inputs[0].dtype, name="type_input"),    
            tf.TensorSpec([batch_size] + model.inputs[1].shape[1:], model.inputs[0].dtype, name="class_input"))   
    real_model = tf.function(model).get_concrete_function(argument)
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(real_model)

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                            run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops
    

In [ ]:
batch_size=32
fold = 1
folderPath = configs["FOLDER_PATH"] + str(fold) + "/"
model = ModelHandler.loadModel(folderPath + 'best_model.h5')

Lets include the output network:

In [ ]:
flops_DFML = get_flops(model, 32)

print(f"DFML FLOPS: {flops_DFML / 10**9:.04} G")



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
DFML FLOPS: 7.643 G


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 16640, 1)]   0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 16632, 60)    600         input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_10 (LeakyReLU)      (None, 16632, 60)    0           conv1d_5[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_5 (MaxPooling1D)  (None, 4158, 60)     0           leaky_re_lu_10[0][0]             
____________________________________________________________________________________________